# Train NN for Wind Speed Prediction

In this notebook we are using Google Colab in order to train a simple shallow Neural Network with 128 input units, 64 FC units and 7 output units. Our NN model will be built using Keras, a neural network API built on top of TensorFlow deep learning framework. 




# Preliminaries

## Import libraries 

In [0]:
# Ignoring unnecessory warnings
import warnings
warnings.filterwarnings("ignore") 
# For input-output handling
import io 
# For large and multi-dimensional arrays
import numpy as np
# For data manipulation and analysis
import pandas as pd
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

## Is GPU working?

In [2]:
import tensorflow as tf
if tf.test.gpu_device_name():
  from tensorflow.python.client import device_lib
  print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14106748275340073586
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 5661722766817518622
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 2009836054129689277
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15956161332
locality {
  bus_id: 1
  links {
  }
}
incarnation: 9373963334984700366
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
]


# Import datasets

## PyDrive Client

In [0]:
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

## Upload files 

### Upload nn_representations.csv

In [4]:
link = input("Provide link for the nn_representations.csv: ")

Provide link for the nn_representations.csv: https://drive.google.com/open?id=12rlY3YjzDzI5AtDYDda05OvSplARWNg8


In [5]:
# Verify that you have everything after '='
fluff, id = link.split('=')
print (id) 

12rlY3YjzDzI5AtDYDda05OvSplARWNg8


In [0]:
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('nn_representations.csv')  
X = pd.read_csv('nn_representations.csv', encoding='utf-8', sep=',')
# Dataset is now stored in a Pandas Dataframe

### Upload actual.csv

In [7]:
link = input("Provide link for the actual.csv: ")

Provide link for the actual.csv: https://drive.google.com/open?id=1NUDBMzyjoCdw64UdjEbhO1S7TFndQrZZ


In [8]:
# Verify that you have everything after '='
fluff, id = link.split('=')
print (id) 

1NUDBMzyjoCdw64UdjEbhO1S7TFndQrZZ


In [0]:
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('actual.csv')  
Y = pd.read_csv('actual.csv', encoding='utf-8', sep=',')
# Dataset is now stored in a Pandas Dataframe

# Display datasets
In order to gain a basic intuition for our datasets, it is of utter importance to take a look on it. In applications like wind speed predictions, it is logical to not understand what kinds of values exists and what is their importance. However, we should never surpass this step.  

## Display nn_representations.csv

In [10]:
dynamic_columns = []
for i in range(X.shape[1] - 1):
    dynamic_columns.append('X_' + str(i+1))
X.columns = ['Timestamp'] + dynamic_columns
with pd.option_context('display.max_rows', 10, 'display.max_columns', None): 
    display(X)

,Timestamp,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,X_11,X_12,X_13,X_14,X_15,X_16,X_17,X_18,X_19,X_20,X_21,X_22,X_23,X_24,X_25,X_26,X_27,X_28,X_29,X_30,X_31,X_32,X_33,X_34,X_35,X_36,X_37,X_38,X_39,X_40,X_41,X_42,X_43,X_44,X_45,X_46,X_47,X_48,X_49,X_50,X_51,X_52,X_53,X_54,X_55,X_56,X_57,X_58,X_59,X_60,X_61,X_62,X_63,X_64,X_65,X_66,X_67,X_68,X_69,X_70,X_71,X_72,X_73,X_74,X_75,X_76,X_77,X_78,X_79,X_80,X_81,X_82,X_83,X_84,X_85,X_86,X_87,X_88,X_89,X_90,X_91,X_92,X_93,X_94,X_95,X_96,X_97,X_98,X_99,X_100,X_101,X_102,X_103,X_104,X_105,X_106,X_107,X_108,X_109,X_110,X_111,X_112,X_113,X_114,X_115,X_116,X_117,X_118,X_119,X_120,X_121,X_122,X_123,X_124,X_125,X_126,X_127,X_128
0,2016-04-01 06:00:00,0.222760,0.031341,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.012391,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.070948,0.0,0.0,0.000000,0.000000,0.0,0.080119,0.0,0.0,0.063162,0.0,0.178438,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.059373,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.170130,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.130505,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.011891,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.004412,0.0,0.000000,0.018352,0.0,0.0,0.191468,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.194363,0.0,0.0,0.0,0.0,0.000000,0.000000
1,2016-04-01 07:00:00,0.287563,0.057565,0.0,0.0,0.014476,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.037809,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.005700,0.0,0.0,0.0,0.0,0.102523,0.0,0.0,0.000000,0.000000,0.0,0.080410,0.0,0.0,0.129742,0.0,0.228498,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.100393,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.223172,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.155867,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.020527,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.065723,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.025352,0.000000,0.0,0.0,0.247759,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.259938,0.0,0.0,0.0,0.0,0.021556,0.000000
2,2016-04-01 08:00:00,0.316299,0.077519,0.0,0.0,0.040638,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.059886,0.000000,0.000000,0.0,0.000000,0.028825,0.0,0.0,0.0,0.0,0.0,0.081773,0.0,0.0,0.0,0.0,0.142289,0.0,0.0,0.000000,0.000000,0.0,0.072181,0.0,0.0,0.202706,0.0,0.260160,0.0,0.0,0.0,0.0,0.042358,0.0,0.0,0.154696,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.279837,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.188829,0.000000,0.0,0.0,0.0,0.014948,0.000000,0.066501,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.115520,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.082848,0.000000,0.0,0.0,0.275202,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.304043,0.0,0.0,0.0,0.0,0.070380,0.000000
3,2016-04-01 09:00:00,0.311988,0.030557,0.0,0.0,0.069818,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.043399,0.000000,0.000000,0.0,0.000000,0.049003,0.0,0.0,0.0,0.0,0.0,0.176186,0.0,0.0,0.0,0.0,0.173265,0.0,0.0,0.000000,0.000000,0.0,0.019146,0.0,0.0,0.261293,0.0,0.251386,0.0,0.0,0.0,0.0,0.081998,0.0,0.0,0.191395,0.000000,0.019097,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.313255,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.201590,0.000000,0.0,0.0,0.0,0.043545,0.000000,0.094460,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.152126,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.001973,0.0,0.000000,0.0,0.114974,0.039670,0.0,0.0,0.299487,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.296824,0.0,0.0,0.0,0.0,0.084066,0.000000
4,2016-04-01 10:00:00,0.296113,0.000000,0.0,0.0,0.110641,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.008745,0.000000,0.000000,0.0,0.000000,0.050617,0.0,0.0,0.0,0.0,0.0,0.276430,0.0,0.0,0.0,0.0,0.193106,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.317937,0.0,0.226482,0.0,0.0,0.0,0.0,0.111199,0.0,0.0,0.192580,0.000000,

## Display actual.csv

In [11]:
dynamic_columns = []
for i in range(Y.shape[1] - 1):
    dynamic_columns.append('X_' + str(i+1))
Y.columns = ['Timestamp'] + dynamic_columns
with pd.option_context('display.max_rows', 10, 'display.max_columns', None): 
    display(Y)

,Timestamp,X_1,X_2,X_3,X_4,X_5,X_6,X_7
0,2016-04-01 06:00:00,0.140067,0.171467,0.148970,0.149539,0.126694,0.110507,0.152039
1,2016-04-01 07:00:00,0.151794,0.166229,0.114302,0.147076,0.126694,0.138523,0.153894
2,2016-04-01 08:00:00,0.113355,0.151834,0.103466,0.126768,0.115007,0.133853,0.115576
3,2016-04-01 09:00:00,0.083390,0.118457,0.101301,0.112616,0.115007,0.130739,0.108779
4,2016-04-01 10:00:00,0.097071,0.115838,0.109427,0.110769,0.111317,0.119846,0.094561
...,...,...,...,...,...,...,...,...
23982,2018-12-31 18:00:00,0.588925,0.580495,0.472914,0.583385,0.523373,0.628015,0.527814
23983,2018-12-31 19:00:00,0.607816,0.594895,0.495666,0.606152,0.563963,0.695719,0.549444
23984,2018-12-31 20:00:00,0.633223,0.593587,0.503791,0.644308,0.583642,0.764203,0.564277
23985,2018-12-31 21:00:00,0.631269,0.597514,0.495123,0.627076,0.575033,0.731518,0.545117


In [0]:
# Everything seems fine, so we move forward to train our NN 

# Train NN model